In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import os
from librosa import load,feature
import librosa
from sklearn.preprocessing import StandardScaler

In [2]:
model = load_model('vber_rav_conv1d_82.h5')

In [3]:
def extract_features(data):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

In [4]:
def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data,sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # without augmentation
    res1 = extract_features(data)
    result = np.array(res1)
    

    return result

In [31]:
sample_rate=22050
X = get_features('happy.wav')
x=list([X])

In [32]:
type(x)

list

In [33]:
x = np.array(x)

In [34]:
x.reshape(1,162,-1)
x.shape

(1, 162)

In [35]:
#print('x before reshape',X.shape)
scaler = StandardScaler()
x=scaler.fit_transform(x)

In [36]:
x.shape

(1, 162)

In [37]:
prediction_feature=x.reshape(1,162,-1)
print('x after reshape',x.shape)

x after reshape (1, 162)


In [38]:
p=model.predict(prediction_feature)

In [39]:
print(np.argmax(p))

7


In [40]:
p

array([[4.5923990e-01, 6.1331148e-37, 2.1508877e-09, 3.7587387e-09,
        1.1996447e-14, 3.0016925e-28, 7.4884419e-21, 5.4076010e-01]],
      dtype=float32)